In [ ]:
#Install module 
pip install spotipy

In [2]:
#Import Dependencies 
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import base64
from config import client_id, client_secret
import pandas as pd
import time 
from urllib.parse import urlencode
from pprint import pprint

In [3]:
#Sign in thru Spotidy credtentials from dashboard 
client_credentials_manager = SpotifyClientCredentials(client_id,client_secret)

#Authorize credentials with token 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
#Query for playlist ID 
result_one = sp.search('This is Rich Brian', type='playlist',limit='03')
pprint(result_one)


{'playlists': {'href': 'https://api.spotify.com/v1/search?query=This+is+Rich+Brian&type=playlist&offset=0&limit=3',
               'items': [{'collaborative': False,
                          'description': 'All the essentials from Rich Brian, '
                                         'right here.',
                          'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX0eg5lGPtFS1'},
                          'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX0eg5lGPtFS1',
                          'id': '37i9dQZF1DX0eg5lGPtFS1',
                          'images': [{'height': None,
                                      'url': 'https://i.scdn.co/image/ab67706f000000035f96d4d26c12baaf88a7f691',
                                      'width': None}],
                          'name': 'This Is Rich Brian',
                          'owner': {'display_name': 'Spotify',
                                    'external_urls': {'spotify': 'https://open.spotify

In [5]:
#Defauly spotify profile id 
Spotify_profile_id = 'cef03df234f441af'

In [6]:
#Extract playlist id for top artist 
rich_playlist_id = '37i9dQZF1DX0eg5lGPtFS1'

In [7]:
#Create function to get indiidual track id 
def getTrackID(user,playlist_id):
    #Create a empty list to store ID's 
    id = []
    play_list = sp.user_playlist(user,playlist_id)

    #Loop thru ID's
    for item in play_list['tracks']['items']:
        track = item['track']
        id.append(track['id'])
    return id 


In [8]:
# ('spotify_id','playlist_id')
id = getTrackID(Spotify_profile_id,rich_playlist_id)

In [9]:
#Create function to get ID features 
def getTrackFeatures(id):

    #Query spotify track and feature attribute
    info = sp.track(id)
    features = sp.audio_features(id)

    #Append info and features for specific track ID's
    name = info['name']
    album = info['album']['name']
    artist = info['album']['artists'][0]['name']
    release_date = info['album']['release_date']
    length = info['duration_ms']
    popularity = info['popularity']


    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    livenss = features[0]['liveness']
    tempo = features[0]['tempo']
    valence = features[0]['valence']

    track = [name,album,artist,release_date,length,popularity,acousticness,danceability,energy,livenss,tempo,valence]

    return track



In [10]:
#Create dataframe and loop thru each track ID features to PD
rich_playlist = []
for i in range(len(id)):
    song = getTrackFeatures(id[i])
    rich_playlist.append(song)

df = pd.DataFrame(rich_playlist,columns = ['song name','album','artist','release_date','length(ms)','popularity','acousticness','danceability','energy','liveness','tempo','valence'])

#Export dataframe to CSV
df.to_csv('rich_brian.csv', sep = ',')
    

In [11]:
#Display dataframe 
df.head(10)

,song name,album,artist,release_date,length(ms),popularity,acousticness,danceability,energy,liveness,tempo,valence
0,New Tooth,New Tooth,Rich Brian,2021-11-04,220000,50,0.63100,0.788,0.678,0.1410,120.019,0.390
1,Run It (feat. Rick Ross & Rich Brian),Shang-Chi and The Legend of The Ten Rings: The...,88rising,2021-09-03,163373,68,0.01950,0.688,0.725,0.0726,104.006,0.225
2,Lazy Susan (with Rich Brian feat. Warren Hue &...,Shang-Chi and The Legend of The Ten Rings: The...,88rising,2021-09-03,279960,51,0.04280,0.766,0.825,0.1020,155.967,0.646
3,edamame (feat. Rich Brian),edamame (feat. Rich Brian),bbno$,2021-07-24,133706,75,0.02300,0.815,0.848,0.0265,106.032,0.684
4,Sydney,Sydney,Rich Brian,2021-03-16,194240,46,0.47500,0.620,0.476,0.1350,161.168,0.602
5,DOA,1999,Rich Brian,2020-08-24,161162,45,0.22700,0.603,0.611,0.1090,86.001,0.317
6,Don't Care,Don't Care,Rich Brian,2020-08-18,164571,43,0.18000,0.642,0.569,0.1050,140.095,0.194
7,Love In My Pocket,Love In My Pocket,Rich Brian,2020-07-09,223093,41,0.02060,0.829,0.597,0.1100,113.011,0.782
8,BALI,BALI,Rich Brian,2020-04-09,199384,39,0.00983,0.864,0.642,0.2200,129.995,0.622
9,TITANIC (feat. Rich Brian),MIRRORS,Jackson Wang,2019-10-25,141146,54,0.04630,0.894,0.748,0.0570,99.983,0.697
